<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good at cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [37]:
# Import libraries here.
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

The `os` library ([https://docs.python.org/3/library/os.html](https://docs.python.org/3/library/os.html)) is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [38]:
# Create an output folder in master folder
# output = '/Users/aidancurley/Documents/dsir/Submissions/Labs/2.02-lab-excel-madness-master/output'
# os.mkdir(output)

In [66]:
# Create a files variable that contains all of our data files.
files = os.listdir('/Users/aidancurley/Documents/dsir/Submissions/Labs/2.02-lab-excel-madness-master/data')

In [67]:
files

['Jan 23.xlsx',
 'Jan 19.xlsx',
 'Jan 15.xlsx',
 'Jan 7.xlsx',
 'Jan 6.xlsx',
 'Jan 14.xlsx',
 'Jan 18.xlsx',
 'Jan 22.xlsx',
 'Jan 29.xlsx',
 'Jan 13.xlsx',
 'Jan 1.xlsx',
 'Jan 25.xlsx',
 'Jan 24.xlsx',
 'Jan 12.xlsx',
 'Jan 28.xlsx',
 'Jan 3.xlsx',
 'Jan 11.xlsx',
 'Jan 27.xlsx',
 'Jan 31.xlsx',
 'Jan 30.xlsx',
 'Jan 26.xlsx',
 'Jan 10.xlsx',
 'Jan 2.xlsx',
 'Jan 9.xlsx',
 'Jan 21.xlsx',
 'Jan 5.xlsx',
 'Jan 17.xlsx',
 'Jan 16.xlsx',
 'Jan 4.xlsx',
 'Jan 20.xlsx',
 'Jan 8.xlsx']

### Step 2: Process one data frame (one sheet from one file)
It looks like we have data for the month of January--31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.) to take a look at what you'll be reading in.

In [68]:
# Read in data from only ONE city (your city or one city of choice) from January 1st.

# https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook

# How to read in an excel file
xls = pd.ExcelFile('/Users/aidancurley/Documents/dsir/Submissions/Labs/2.02-lab-excel-madness-master/data/Jan 1.xlsx')

In [69]:
# How to get a specific sheet in an excel file
df_1= pd.read_excel(xls, 'New York')

In [70]:
# 129 Rows, 4 Columns
# Format is the same for every sheet in every file
df_1.shape

(129, 4)

### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [71]:
df_1['price_eu'][0]

2.0481412084773183

In [72]:
# Convert eu to usd
df_1['price_usd'] = df_1['price_eu'] * 1.1

In [73]:
# Convert kg to lb
df_1['weight_lb'] = df_1['weight_kg'] * 2.2

In [74]:
df_1.head(1)

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb
0,4159,2.048141,8.812961,138,2.252955,19.388515


### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data from above.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [75]:
# Read in the plu-codes.csv file
plu = pd.read_csv("plu-codes.csv")

In [76]:
plu.head(1)

,product,plu_code
0,Apple (Fuji),4131


In [77]:
# Rename the column plu_code so it can match the prodcode in the df_1 dataframe
plu.rename(columns={'plu_code': 'prodcode'}, inplace=True)

In [78]:
plu.head(1)

,product,prodcode
0,Apple (Fuji),4131


In [79]:
df_1.head(1)

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb
0,4159,2.048141,8.812961,138,2.252955,19.388515


In [80]:
# Merge df_1 and plu on the 'prodcode' columns
jan_1 = pd.merge(df_1, plu, on='prodcode')

In [81]:
jan_1.head()

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb,product
0,4159,2.048141,8.812961,138,2.252955,19.388515,Onion
1,4159,2.124963,1.906494,319,2.337459,4.194287,Onion
2,4159,1.834220,7.788227,229,2.017642,17.134099,Onion
3,4159,1.281120,4.506284,164,1.409232,9.913825,Onion
4,4159,1.162582,8.225310,447,1.278840,18.095683,Onion


### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [82]:
# Drop the weight_kg column because it is not necessary anymore
jan_1.drop('weight_kg', axis=1, inplace=True)

In [83]:
jan_1.columns

Index(['prodcode', 'price_eu', 'quantity', 'price_usd', 'weight_lb',
       'product'],
      dtype='object')

In [84]:
# Drop the price_eu column because it is not necessary anymore
jan_1.drop('price_eu', axis=1, inplace=True)

In [85]:
jan_1.columns

Index(['prodcode', 'quantity', 'price_usd', 'weight_lb', 'product'], dtype='object')

### Step 2d: Add the date
Simply create a new `date` column for this dataframe that contains the date this data was collected. For example, this data is from `Jan 1.xlsx`, so this column should be full of `Jan 1`.  
_Note: In this step, this does not need to be an automated function that would assign the relevant date for data from any file. Simply create this one column._

In [86]:
jan_1['date'] = 'Jan 1'

In [87]:
jan_1.head(2)

,prodcode,quantity,price_usd,weight_lb,product,date
0,4159,138,2.252955,19.388515,Onion,Jan 1
1,4159,319,2.337459,4.194287,Onion,Jan 1


### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [140]:
def process_data(file, city):
    # 2 Process one data frame
    path = '/Users/aidancurley/Documents/dsir/Submissions/Labs/2.02-lab-excel-madness-master/data/'+ file
    new_xls = pd.ExcelFile(path)
    new_df = pd.read_excel(new_xls, city)
    
    # 2a Convert to 'Merican columns
    new_df['price_usd'] = new_df['price_eu'] * 1.1
    new_df['weight_lb'] = new_df['weight_kg'] * 2.2
    
    # 2b Merge in product names
    plu = pd.read_csv("plu-codes.csv")
    plu.rename(columns={'plu_code': 'prodcode'}, inplace=True)
    new_df = pd.merge(new_df, plu, on='prodcode')
    
    #2c Drop unnecessary columns
    new_df.drop(columns=['weight_kg', 'price_eu'], axis=1, inplace=True) 
    
    #2d Add the date
    new_df['date'] = file[:-5]
    
    return new_df

Test your function out on a new file and city (one sheet from that file)!

In [141]:
process_data('Jan 10.xlsx', 'Atlanta')

,prodcode,quantity,price_usd,weight_lb,product,date
0,4225,121,1.321429,21.207317,Avocado,Jan 10
1,4225,164,2.776709,17.974300,Avocado,Jan 10
2,4225,120,1.665817,10.835232,Avocado,Jan 10
3,4225,455,1.571825,17.214646,Avocado,Jan 10
4,4225,418,1.634974,2.765967,Avocado,Jan 10
...,...,...,...,...,...,...
132,4062,404,1.486384,13.197261,Cucumber,Jan 10
133,3302,348,2.376736,3.499581,Apricot,Jan 10
134,3302,211,0.619958,6.331263,Apricot,Jan 10
135,3302,423,0.910686,6.511946,Apricot,Jan 10


In [142]:
files

['Jan 23.xlsx',
 'Jan 19.xlsx',
 'Jan 15.xlsx',
 'Jan 7.xlsx',
 'Jan 6.xlsx',
 'Jan 14.xlsx',
 'Jan 18.xlsx',
 'Jan 22.xlsx',
 'Jan 29.xlsx',
 'Jan 13.xlsx',
 'Jan 1.xlsx',
 'Jan 25.xlsx',
 'Jan 24.xlsx',
 'Jan 12.xlsx',
 'Jan 28.xlsx',
 'Jan 3.xlsx',
 'Jan 11.xlsx',
 'Jan 27.xlsx',
 'Jan 31.xlsx',
 'Jan 30.xlsx',
 'Jan 26.xlsx',
 'Jan 10.xlsx',
 'Jan 2.xlsx',
 'Jan 9.xlsx',
 'Jan 21.xlsx',
 'Jan 5.xlsx',
 'Jan 17.xlsx',
 'Jan 16.xlsx',
 'Jan 4.xlsx',
 'Jan 20.xlsx',
 'Jan 8.xlsx']

### Step 4: Process all of January's data for your city
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [143]:
jan_ny = [process_data(i, 'New York') for i in files]

In [145]:
# Each index is a pandas dataframe
jan_ny[0]

,prodcode,quantity,price_usd,weight_lb,product,date
0,4650,109,1.330281,17.027600,Mushroom,Jan 23
1,4650,107,1.979061,3.396520,Mushroom,Jan 23
2,4650,498,2.739112,5.918647,Mushroom,Jan 23
3,4650,350,1.092037,19.737597,Mushroom,Jan 23
4,4650,216,0.778506,21.484154,Mushroom,Jan 23
...,...,...,...,...,...,...
155,4309,288,1.485246,4.698512,Lychee,Jan 23
156,4309,210,1.728817,8.737249,Lychee,Jan 23
157,4309,148,2.619922,16.792627,Lychee,Jan 23
158,4309,198,1.830828,19.490797,Lychee,Jan 23


### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [94]:
# https://stackoverflow.com/questions/32444138/concatenate-a-list-of-pandas-dataframes-together
# Fairly simple
ny_merged = pd.concat(jan_ny)

In [95]:
ny_merged

,prodcode,quantity,price_usd,weight_lb,product,date
0,4650,109,1.330281,17.027600,Mushroom,Jan 23
1,4650,107,1.979061,3.396520,Mushroom,Jan 23
2,4650,498,2.739112,5.918647,Mushroom,Jan 23
3,4650,350,1.092037,19.737597,Mushroom,Jan 23
4,4650,216,0.778506,21.484154,Mushroom,Jan 23
...,...,...,...,...,...,...
107,4272,161,1.372635,19.997432,Grapes,Jan 8
108,4240,347,2.565107,13.588784,Blueberries,Jan 8
109,4240,279,2.526195,11.874177,Blueberries,Jan 8
110,4240,164,1.955288,19.035519,Blueberries,Jan 8


### Step 6: Do this for all cities, write data to your output folder
Here's the big one. For each city, process the data as in steps 3-5, and then write the data to your `output` folder. Below is a dictionary of city names to desired output file names.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [96]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [ ]:
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!


In [168]:
# Easy to loop through a list
city_list = ['Atlanta', 'Austin', 'Boston', 'Chicago', 'Denver', 'Los Angeles', 'New York', 'San Francisco', 'Seattle', 'Washington, DC']

In [154]:
city_dict

{'Atlanta': 'atl.csv',
 'Austin': 'atx.csv',
 'Boston': 'bos.csv',
 'Chicago': 'chi.csv',
 'Denver': 'den.csv',
 'Los Angeles': 'lax.csv',
 'New York': 'nyc.csv',
 'San Francisco': 'sf.csv',
 'Seattle': 'sea.csv',
 'Washington, DC': 'dc.csv'}

In [156]:
# How to get the value in city_dict
city_dict[city_list[0]]

'atl.csv'

In [173]:
# Files gives us a string
files[0]

'Jan 23.xlsx'

In [157]:
# This is what I need to use for os.path.join
output

'/Users/aidancurley/Documents/dsir/Submissions/Labs/2.02-lab-excel-madness-master/output'

In [174]:
for i in city_list:
    print(i)

Atlanta
Austin
Boston
Chicago
Denver
Los Angeles
New York
San Francisco
Seattle
Washington, DC


In [177]:
# https://stackoverflow.com/questions/52521594/adding-a-path-to-pandas-to-csv-function
# Credit to David Lee for contributing to this
for _, xi in enumerate(city_list):
    jan_ny = [process_data(n, xi) for n in files]
    concat = pd.concat(jan_ny)
    concat['city'] = xi
    concat = concat.loc[:, ["city", "date", "product", "prodcode", "quantity", "weight_lb", "price_usd"]]
    final_name = city_dict[city_list[_]]
    output_file = os.path.join(output, final_name)
    concat.to_csv(output_file)

In [200]:
jan_ny[1]

,prodcode,quantity,price_usd,weight_lb,product,date
0,4323,272,1.373113,16.080387,Strawberries,Jan 19
1,4323,479,1.550520,4.738151,Strawberries,Jan 19
2,4323,116,2.377354,7.544380,Strawberries,Jan 19
3,4323,232,1.881809,13.793520,Strawberries,Jan 19
4,4225,242,3.298338,9.990708,Avocado,Jan 19
...,...,...,...,...,...,...
108,4309,405,1.202293,5.352873,Lychee,Jan 19
109,4309,273,0.861107,18.502841,Lychee,Jan 19
110,4650,482,1.841775,5.635449,Mushroom,Jan 19
111,4650,369,1.198938,10.979843,Mushroom,Jan 19


In [198]:
concat

,city,date,product,prodcode,quantity,weight_lb,price_usd
0,"Washington, DC",Jan 23,Orange,4012,488,14.061532,1.724722
1,"Washington, DC",Jan 23,Orange,4012,454,4.301680,1.414691
2,"Washington, DC",Jan 23,Orange,4012,248,19.711042,2.136255
3,"Washington, DC",Jan 23,Orange,4012,295,8.214318,0.934342
4,"Washington, DC",Jan 23,Orange,4012,374,3.933965,1.200956
...,...,...,...,...,...,...,...
163,"Washington, DC",Jan 8,Banana,4011,187,17.033067,0.626851
164,"Washington, DC",Jan 8,Banana,4011,476,9.309348,3.126045
165,"Washington, DC",Jan 8,Banana,4011,238,5.209897,0.882481
166,"Washington, DC",Jan 8,Banana,4011,118,12.531757,1.554881


# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list _(Hint: Using the `os` library and a list comprehension would make this 1-2 lines of code!)_
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [178]:
import glob

In [180]:
output

'/Users/aidancurley/Documents/dsir/Submissions/Labs/2.02-lab-excel-madness-master/output'

In [181]:
# https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
all_files = glob.glob(output + "/*.csv")
df = pd.concat((pd.read_csv(f) for f in all_files))

In [183]:
df.head()

,Unnamed: 0,city,date,product,prodcode,quantity,weight_lb,price_usd
0,0,Los Angeles,Jan 23,Lemon,4958,263,12.163276,2.880092
1,1,Los Angeles,Jan 23,Lemon,4958,222,21.167025,1.486121
2,2,Los Angeles,Jan 23,Lemon,4958,355,13.458449,2.237626
3,3,Los Angeles,Jan 23,Lemon,4958,101,11.642977,1.303736
4,4,Los Angeles,Jan 23,Lemon,4958,185,10.649555,1.493155


In [197]:
df

,Unnamed: 0,city,date,product,prodcode,quantity,weight_lb,price_usd
0,0,Los Angeles,Jan 23,Lemon,4958,263,12.163276,2.880092
1,1,Los Angeles,Jan 23,Lemon,4958,222,21.167025,1.486121
2,2,Los Angeles,Jan 23,Lemon,4958,355,13.458449,2.237626
3,3,Los Angeles,Jan 23,Lemon,4958,101,11.642977,1.303736
4,4,Los Angeles,Jan 23,Lemon,4958,185,10.649555,1.493155
...,...,...,...,...,...,...,...,...
4585,155,Boston,Jan 8,Apple (Fuji),4131,445,21.162546,2.201526
4586,156,Boston,Jan 8,Apple (Fuji),4131,409,10.426867,0.681161
4587,157,Boston,Jan 8,Eggplant,4081,211,9.991937,2.813027
4588,158,Boston,Jan 8,Eggplant,4081,453,14.006035,1.422620


In [189]:
# https://towardsdatascience.com/pandas-pivot-the-ultimate-guide-5c693e0771f3 
# ^ Helped with figuring out setup for this pivot table
pivot_df = df.pivot_table(index="city", values=["quantity", "price_usd", "weight_lb"], aggfunc=np.mean)

In [195]:
pivot_df = pivot_df.loc[:, ["quantity", "weight_lb", "price_usd"]]

In [196]:
pivot_df

,quantity,weight_lb,price_usd
city,,,
Atlanta,302.485306,11.929027,1.937980
Austin,301.048298,12.092775,1.937456
Boston,298.806536,12.063057,1.900973
Chicago,301.686186,12.101555,1.930026
Denver,301.012745,12.129850,1.932088
Los Angeles,301.531276,12.167527,1.918331
New York,299.482863,12.090294,1.912662
San Francisco,298.979895,12.237399,1.925830
Seattle,300.333694,11.925486,1.903315


# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file (editable in any text editor) and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - **Comment your code.** Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).